In [2]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/facial_expression_detection/research'

In [3]:
os.chdir('../')
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/facial_expression_detection'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    dataset_folder: Path
    transformed_dataset: Path
    dataset_labels_src: Path
    dataset_labels: Path
    median_filter_size: int

In [5]:
from src.detmood.constant import *
from src.detmood.utils.main_utils import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        params = self.params.transform.noise_reduction
        
        create_directories([config.transformed_dataset])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            dataset_folder=config.dataset_folder,
            transformed_dataset=config.transformed_dataset,
            dataset_labels_src=config.dataset_labels_src,
            dataset_labels=config.dataset_labels,
            median_filter_size=params.median_filter_size
        )
        
        return data_transformation_config

In [8]:
import cv2
from tqdm import tqdm
import pandas as pd

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def equalize_histogram(self, img):
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        img_hsv[:,:,2] = cv2.equalizeHist(img_hsv[:,:,2])
        img_eq = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
        
        return img_eq
    
    def noise_reduction(self, img):
        img_filt = cv2.medianBlur(img, self.config.median_filter_size)
        
        return img_filt
    
    def labels_csv_transform(self):
        labels_df = pd.read_csv(self.config.dataset_labels_src)
        
        for ind in labels_df.index:
            labels_df.loc[ind, 'label'] = MOOD_DICT[labels_df.loc[ind, 'label']]
        
        labels_df.to_csv(self.config.dataset_labels, index=False)
    
    def transformation_compose(self):
        for img_name in tqdm(os.listdir(self.config.dataset_folder)):
            img = cv2.imread(os.path.join(self.config.dataset_folder, img_name))
            img_eq = self.equalize_histogram(img)
            img_filt = self.noise_reduction(img_eq)
            
            cv2.imwrite(os.path.join(self.config.transformed_dataset, img_name), img_filt)
        
        self.labels_csv_transform()

In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transformation_compose()

except Exception as e:
    raise e

[2024-10-27 10:27:39,328: INFO: main_utils: created directory at: artifacts]
[2024-10-27 10:27:39,329: INFO: main_utils: created directory at: artifacts/data_transformation/dataset]


100%|██████████| 10759/10759 [00:04<00:00, 2344.10it/s]
